In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import (Input, Dense)

## Load data to tuple list

In [29]:
import os 
import glob

audio_len = 1500

if os.name == "nt":
    os.chdir("C:\\Users\\tymons\\001.Projects\\003.eul\\workspace\\smartula-analysis\\core\\csv")
else:
    os.chdir("/home/tymons/Projects/003.eul/workspace/smartula-analysis/csv")
    

all_filenames = [i for i in glob.glob("*.{}".format("csv"))]
#all_filenames = all_filenames[:600]
list_of_audios = []
for filename in all_filenames:
    samples = np.ravel(pd.read_csv(filename, header=None))
    array = np.array(samples[:audio_len]).astype(float)
    samples = array - array.mean()
    audio_tuple = (filename, samples)
    if len(samples) == audio_len:
        list_of_audios.append(audio_tuple)
    else:
        print("Error with: " + filename +". Got length:" + str(len(samples)))


Error with: 2019-06-06T03-23-11.csv. Got length:1000
Error with: 2019-06-07T03-09-11.csv. Got length:1000


## Model

In [3]:
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

sound_input = Input(shape=(1500,))
encoded = Dense(784, activation='relu')(sound_input)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='relu')(decoded)
decoded = Dense(1500, activation='relu')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(sound_input, decoded)

W0626 13:13:37.020771 140145306703616 deprecation_wrapper.py:119] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0626 13:13:37.050802 140145306703616 deprecation_wrapper.py:119] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 13:13:37.054440 140145306703616 deprecation_wrapper.py:119] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [5]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               1176784   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
__________

In [4]:
encoder = Model(sound_input, encoded)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               1176784   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
Total params: 1,287,600
Trainable params: 1,287,600
Non-trainable params: 0
_________________________________________________________________


In [6]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

W0626 13:13:48.500923 140145306703616 deprecation_wrapper.py:119] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0626 13:13:48.526515 140145306703616 deprecation_wrapper.py:119] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0626 13:13:48.533036 140145306703616 deprecation.py:323] From /home/tymons/Projects/003.eul/workspace/smartula-analysis/venv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [30]:
k = 0.8

data = np.array([audio[1] for audio in list_of_audios])
print(data.shape)

index = int(k*len(data))
x_train = data[:index]
x_test = data[index:]
print(x_train.shape)
print(x_test.shape)


(1069, 1500)
(855, 1500)
(214, 1500)


In [148]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                shuffle=True,
                validation_data=(x_test, x_test))


ValueError: Error when checking input: expected input_1 to have shape (1500,) but got array with shape (1,)

In [ ]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


In [ ]:
%matplotlib inline

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()